# Builder Pattern
- some objects are simple and can be created in single initalizer call
- Other require lot of ceremony to create
- Having an object with 10 initalizer arguments is not productive
- Instead opt for piecewise construction
- builder provides an API for constructing an object step-by-step


In [1]:
text = 'hello'
parts = ['<p>',text,'</p>']
print("".join(parts))

<p>hello</p>


Trying to build a html file with paragraphs

In [2]:
words = ['hello', 'world']
parts = ['<ul>']
for w in words:
    parts.append(f'<li>{w}</li>')
parts.append('</ul')
print("\n".join(parts))

<ul>
<li>hello</li>
<li>world</li>
</ul


In [3]:
class html_element:
    indent_size = 2

    def __init__(self, name='', text=''):
        self.text = text
        self.name = name
        self.elements = []
    
    def __str(self, indent):
        lines = []
        i = ' ' * (indent * self.indent_size)
        lines.append(f'{i}<{self.name}>')

        if self.text:
            i1 = ' ' * ((indent + 1) * self.indent_size)
            lines.append(f'{i1}{self.text}')

        for e in self.elements:
            lines.append(e.__str(indent + 1))

        lines.append(f'{i}</{self.name}>')
        return '\n'.join(lines)
    
    def __str__(self):
        return self.__str(0)
    
    @staticmethod
    def create(name):
        return html_builder(name)

In [4]:
class html_builder:
    def __init__(self, root_name):
        self.root_name = root_name
        self.__root = html_element(root_name)
    
    def add_child(self, child_name, child_text):
        self.__root.elements.append(
            html_element(child_name, child_text)
        )
    
    def add_child_fluent(self, child_name, child_text):
        self.__root.elements.append(
            html_element(child_name, child_text)
        )
        return self

    def __str__(self):
        return str(self.__root)


In [5]:
builder = html_builder('ul')
builder.add_child('li','hello')
builder.add_child('li','world')
print("---------------")
print('Builder')
print("---------------")
print(builder)
print("---------------")

---------------
Builder
---------------
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>
---------------


In [7]:
builder = html_builder('ul')
builder.add_child_fluent('li','hello').\
    add_child_fluent('li','world')
print("---------------")
print('Builder')
print("---------------")
print(builder)
print("---------------")

---------------
Builder
---------------
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>
---------------


In [8]:
builder = html_element.create('ul')
builder.add_child('li','hello')
builder.add_child('li','world')
print("---------------")
print('Builder')
print("---------------")
print(builder)
print("---------------")

---------------
Builder
---------------
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>
---------------


In [9]:
class Person:
    def __init__(self):
        # address
        self.street_number = None
        self.post_code = None
        self.city = None

        # employment
        self.company = None
        self.position = None
        self.annual_income = None
    
    def __str__(self):
        return f'Address: {self.street_number}, {self.post_code}, {self.city} ' + \
                f'Employed at {self.company} as {self.position} and earning {self.annual_income}'


In [10]:
class PersonBuilder:
    def __init__(self, person=Person()):
        if person is None:
            self.person = Person()
        else:
            self.person = person
    
    @property
    def lives(self):
        return PersonAddressBuilder(self.person)

    @property
    def works(self):
        return PersonJobBuilder(self.person)
    
    def build(self):
        return self.person

In [11]:
class PersonJobBuilder(PersonBuilder):
    def __init__(self, person):
        super().__init__(person)
    
    def at(self, company_name):
        self.person.company = company_name
        return self
    
    def as_a(self, position):
        self.person.position = position
        return self
    
    def earning(self, annual_income):
        self.person.annual_income = annual_income
        return self

In [12]:
class PersonAddressBuilder(PersonBuilder):
    def __init__(self, person):
        super().__init__(person)

    def at(self, street_address):
        self.person.street_number = street_address
        return self

    def with_postcode(self, postcode):
        self.person.post_code = postcode
        return self

    def in_city(self, city):
        self.person.city = city
        return self

In [13]:
pb = PersonBuilder()
person = pb\
        .lives.at('1234').in_city('cbe').with_postcode('641039')\
        .works.at('ABC Company').as_a('Engineer').earning(1234).build()

print(person)

Address: 1234, 641039, cbe Employed at ABC Company as Engineer and earning 1234


# Factory Pattern
- Object Creation login becomes too convoluted
- Initalizer is not descriptive
    - Cannot overload with same sets of arguments with differnt names
- Wholesale Object creation(non-piecewise, unlike builder) can be outsourced
    - A seperate method(Factory Method)
    - That may exist in a seperate class(Factory)

In [15]:
class Point:
    def __init__(self,x, y):
        self.x = x
        self.y = y
    
    def __init__(self, rho, theta):
        self.rho = rho
        self.theta = theta

cant be done because init can be overloaded with same values

In [16]:
from enum import Enum
from math import *
class CoordinateSystem(Enum):
    CARTESIAN = 1
    POLAR = 2

class Point:
    def __init__(self, a, b, system):
        if system == CoordinateSystem.CARTESIAN:
            self.x = a
            self.y = b
        if system == CoordinateSystem.POLAR:
            self.x = a *cos(b)
            self.y = a *sin(b)

In [17]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __str__(self):
        return f'x: {self.x} y: {self.y}'
    
    @staticmethod
    def new_cartesian_point(x,y):
        return Point(x,y)

    @staticmethod
    def new_polar_point(rho, theta):
        return Point(rho*cos(theta), rho*sin(theta))

In [18]:
p = Point(2,2)
p1 = Point.new_cartesian_point(2,2)
print(p1)
p2 = Point.new_polar_point(2,2)
print(p2)

x: 2 y: 2
x: -0.8322936730942848 y: 1.8185948536513634


In [19]:
class PointFactory:
    @staticmethod
    def new_cartesian_point(x,y):
        return Point(x,y)

    @staticmethod
    def new_polar_point(rho, theta):
        return Point(rho*cos(theta), rho*sin(theta))

p1 = PointFactory.new_cartesian_point(2,2)
print(p1)
p2 = PointFactory.new_polar_point(2,2)
print(p2)

x: 2 y: 2
x: -0.8322936730942848 y: 1.8185948536513634


In [20]:
class Point:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def __str__(self):
        return f'x: {self.x} y: {self.y}'
    class PointFactory:
        def new_cartesian_point(x,y):
            return Point(x,y)

        def new_polar_point( rho, theta):
            return Point(rho*cos(theta), rho*sin(theta))

p1 = Point.PointFactory.new_cartesian_point(2,2)
print(p1)

x: 2 y: 2
